# An Introduction to the Amazon Fraud Detector API  


## Introduction
-------

Amazon Fraud Detector is a fully managed service that makes it easy to identify potentially fraudulent online activities, such as online payment fraud and the creation of fake accounts. 

In this notebook, we'll use the Amazon Fraud Detector API to define an entity and event of interest and use CSV data stored in S3 to train a model. Next, we'll derive some rules and create a "detector" by combining our entity, event, model, and rules into a single endpoint. Finally, we'll apply the detector to a sample of our data to identify potentially fraudulent events.

After running this notebook you should be able to:
- Define an Entity and Event
- Create a Detector
- Train a Machine Learning (ML) Model
- Author Rules to identify potential fraud based on the model's score
- Apply the Detector's "predict" function, to generate a model score and rule outcomes on data

If you would like to know more, please check out [Fraud Detector's Documentation](https://docs.aws.amazon.com/frauddetector/). 


## Setup
------
First setup your AWS credentials so that Fraud Detector can store and access training data and supporting detector artifacts in S3.

Detailed information on setting-up: https://docs.aws.amazon.com/frauddetector/latest/ug/set-up.html

To use Amazon Fraud Detector, you have to set up **permissions** that allow access to the Amazon Fraud Detector console and API operations. You also have to **allow Amazon Fraud Detector to perform tasks on your behalf** and to **access resources that you own**. We recommend creating an AWS Identity and Access Management (IAM) user with access restricted to Amazon Fraud Detector operations and required permissions. You can add other permissions as needed.

The following policies provide the required permission to use Amazon Fraud Detector:

- **AmazonFraudDetectorFullAccessPolicy**  
    Allows you to perform the following actions:  
    - Access all Amazon Fraud Detector resources  
    - List and describe all model endpoints in Amazon SageMaker  
    - List all IAM roles in the account  
    - List all Amazon S3 buckets  
    - Allow IAM Pass Role to pass a role to Amazon Fraud Detector  

- **AmazonS3FullAccess**  
    Allows full access to Amazon S3. This is required to upload training files to S3.  

**IMPORTANT**: each SageMaker notebook needs to assume an **AWS IAM role**. The role of this notebook should include the above 2 policies, and also include in the **Trusted Relationships** the fraud detector service **frauddetector.amazonaws.com**. The final Trusted Relationships part of the role should look similar to this: 

```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": [
          "sagemaker.amazonaws.com",
          "frauddetector.amazonaws.com"
        ]
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```    
------

# Setup and verify your Python environment
Boto3 is the Amazon Web Services (AWS) SDK for Python. It enables Python developers to create, configure, and manage AWS services. You can find more details about Boto3 Fraud Detector [here](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/frauddetector.html). 

In [ ]:
import boto3  # Python library for interacting with AWS resources

fraud_detector_client = boto3.client('frauddetector')  # create a fraud detector client


We now have to define the role and data location that we will be using.

In [ ]:
import sagemaker

ARN_ROLE       = sagemaker.get_execution_role()  # get the current role of the SageMaker notebook (or hardcode an actual role ARN)
S3_BUCKET      = "<your_S3_bucket_name>"
S3_FILE        = "registration_data_20K_full.csv"
S3_FILE_LOC    = "s3://{0}/{1}".format(S3_BUCKET,S3_FILE)

print('Using Role:', ARN_ROLE)
print('Dataset:', S3_FILE_LOC)

# Read and understand the data

We will be using the sample dataset provided in the Amazon Fraud Detector documentation. The dataset must be in a CSV format and uploaded to an S3 bucket in the same region with the Fraud Detector. For more information on the dataset and how to upload it to S3, please check [here](https://docs.aws.amazon.com/frauddetector/latest/ug/step-1-get-s3-data.html).

In [ ]:
import pandas as pd

# connect to S3, get csv file, and convert it to a pandas dataframe
s3_client = boto3.resource('s3')
s3_object = s3_client.Object(S3_BUCKET, S3_FILE)
body = s3_object.get()['Body']
df_data = pd.read_csv(body)

In [ ]:
df_data.head(5)

In [ ]:
df_data.info()  # general info about the data

Our dataframe has **20K records**, and **9 features**. Of these 9 features, 2 are mandatory for Amazon Fraud Detector: ```EVENT_TIMESTAMP``` and ```EVENT_LABEL```. The rest 7 features can be used as **variables** in Amazon Fraud Detector, in order to train a Machine Learning model. 

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('seaborn')
target_variable = df_data.EVENT_LABEL.value_counts()
target_variable.plot.pie(legend=True, autopct='%1.1f%%')
plt.show()
print(target_variable)

As expected, there is a big class imbalance. However, Amazon Fraud Detector can take care of these cases, as long as, fraudulent cases are **more than 400 within a minimum of 10K dataset.**

# Configure Amazon Fraud Detector

## Declare names

In this step we have to declare the name of the variables from the dataset that will be used, as well as, the names of the various components of the Amazon Fraud Detector.

In [ ]:
# declare variables; must match EXACTLY with the actual headers in the csv file!
VARIABLE_IP_ADDRESS = 'ip_address'
VARIABLE_EMAIL_ADDRESS = 'email_address'
VARIABLE_PHONE_NUMBER = 'phone_number'
VARIABLE_USER_AGENT = 'user_agent'
VARIABLE_CUSTOMER_STATE = 'billing_state'
VARIABLE_CUSTOMER_POSTAL = 'billing_postal'
VARIABLE_CUSTOMER_ADDRESS = 'billing_address'

# include the variables that will be used in your model
list_event_variables = [
    VARIABLE_IP_ADDRESS,
    VARIABLE_EMAIL_ADDRESS,
    VARIABLE_PHONE_NUMBER,
    VARIABLE_USER_AGENT,
    VARIABLE_CUSTOMER_STATE,
    VARIABLE_CUSTOMER_POSTAL,
    VARIABLE_CUSTOMER_ADDRESS
]

# declare fraudulent and legitimate labels; must match EXACTLY with the actual EVENT_LABEL value in the csv file!
LABEL_FRAUD = 'fraud'
LABEL_LEGIT = 'legit'

# declare entity name: Entity names must be a-z, all lowercase characters, no spaces (underscores are allowed).
ENTITY_TYPE_NAME = '<your_entity_name>'  # NEED TO BE SAME AS IN THE CONSOLE IF YOU WANT TO USE EXISTING MODELS

# declare event name: Event type name must be a-z, all lowercase characters, no spaces (underscores are allowed).
EVENT_TYPE_NAME = '<your_event_name>'  # NEED TO BE SAME AS IN THE CONSOLE IF YOU WANT TO USE EXISTING MODELS

# declare model name: Model names must be a-z, all lowercase characters, no spaces (underscores are allowed).
MODEL_ID = '<your_model_name>'  # NEED TO BE SAME AS IN THE CONSOLE IF YOU WANT TO USE EXISTING MODELS

# declare detector name: Detector names must be a-z, all lowercase characters, no spaces (underscores are allowed).
DETECTOR_ID = '<your_detector_name>'

# declare outcome names: Outcome names must be a-z, all lowercase characters, no spaces (underscores are allowed).
OUTCOME_BLOCK = '<your_outcome_name1>'
OUTCOME_REVIEW = '<your_outcome_name2>'
OUTCOME_APPROVE = '<your_outcome_name3>'

# declare rule names: Rule names must be a-z, all lowercase characters, no spaces (underscores are allowed).
RULE_ID_HIGH_RISK = '<your_rule_name1>'
RULE_ID_MID_RISK = '<your_rule_name2>'
RULE_ID_LOW_RISK = '<your_rule_name3>'

## Create Variables
Variables are the *features* that we will use in our ML models. We have to choose which of the dataframe columns will be used for that, and we have to **map** them to the specific **variable types** that Amazon Fraud Detector offers. More information about the list of available variable types can be found [here](https://docs.aws.amazon.com/frauddetector/latest/ug/create-a-variable.html#variable-types).

**Important notice**: You cannot create a variable that already exists. If you try to create an already existing variable, you will get a ```ValidationException``` error, highlighting that "Provided variable name already exists in the system". As such, **if you execute the following cell more than one time, you will get this error**. 

In [ ]:
# collect existing variables
response = fraud_detector_client.get_variables()
existing_variable_names = [variable['name'] for variable in response['variables']]

# Create variable email_address
if VARIABLE_EMAIL_ADDRESS not in existing_variable_names:
    fraud_detector_client.create_variable(
        name = VARIABLE_EMAIL_ADDRESS,
        variableType = 'EMAIL_ADDRESS',
        dataSource = 'EVENT',
        dataType = 'STRING',
        defaultValue = '<unknown>'
    )
    print('Created variable:', VARIABLE_EMAIL_ADDRESS)
else: print('Variable', VARIABLE_EMAIL_ADDRESS, 'already exists...')

# Create variable ip_address
if VARIABLE_IP_ADDRESS not in existing_variable_names:
    fraud_detector_client.create_variable(
        name = VARIABLE_IP_ADDRESS,
        variableType = 'IP_ADDRESS',
        dataSource = 'EVENT',
        dataType = 'STRING',
        defaultValue = '<unknown>'
    )
    print('Created variable:', VARIABLE_IP_ADDRESS)
else: print('Variable', VARIABLE_IP_ADDRESS, 'already exists...')


# Create variable billing_state
if VARIABLE_CUSTOMER_STATE not in existing_variable_names:
    fraud_detector_client.create_variable(
        name = VARIABLE_CUSTOMER_STATE,
        variableType = 'BILLING_STATE',
        dataSource = 'EVENT',
        dataType = 'STRING',
        defaultValue = '<unknown>'
    )
    print('Created variable:', VARIABLE_CUSTOMER_STATE)
else: print('Variable', VARIABLE_CUSTOMER_STATE, 'already exists...')

# Create variable user_agent
if VARIABLE_USER_AGENT not in existing_variable_names:
    fraud_detector_client.create_variable(
        name = VARIABLE_USER_AGENT,
        variableType = 'USERAGENT',
        dataSource = 'EVENT',
        dataType = 'STRING',
        defaultValue = '<unknown>'
    )
    print('Created variable:', VARIABLE_USER_AGENT)
else: print('Variable', VARIABLE_USER_AGENT, 'already exists...')

# Create variable billing_postal
if VARIABLE_CUSTOMER_POSTAL not in existing_variable_names:
    fraud_detector_client.create_variable(
        name = VARIABLE_CUSTOMER_POSTAL,
        variableType = 'BILLING_ZIP',
        dataSource = 'EVENT',
        dataType = 'STRING',
        defaultValue = '<unknown>'
    )
    print('Created variable:', VARIABLE_CUSTOMER_POSTAL)
else: print('Variable', VARIABLE_CUSTOMER_POSTAL, 'already exists...')

# Create variable phone_number
if VARIABLE_PHONE_NUMBER not in existing_variable_names:
    fraud_detector_client.create_variable(
        name = VARIABLE_PHONE_NUMBER,
        variableType = 'PHONE_NUMBER',
        dataSource = 'EVENT',
        dataType = 'STRING',
        defaultValue = '<unknown>'
    )
    print('Created variable:', VARIABLE_PHONE_NUMBER)
else: print('Variable', VARIABLE_PHONE_NUMBER, 'already exists...')

# Create variable billing_address
if VARIABLE_CUSTOMER_ADDRESS not in existing_variable_names:
    fraud_detector_client.create_variable(
        name = VARIABLE_CUSTOMER_ADDRESS,
        variableType = 'BILLING_ADDRESS_L1',
        dataSource = 'EVENT',
        dataType = 'STRING',
        defaultValue = '<unknown>'
    )
    print('Created variable:', VARIABLE_CUSTOMER_ADDRESS)
else: print('Variable', VARIABLE_CUSTOMER_ADDRESS, 'already exists...')
    

## Create Labels
Labels are the **ground truth values that describe whether an event is fraudulent or legitimate**. This has to be **exactly** the same word that is used in our CSV file. For example, a fraudulent case could be highlighted as 'fraud', 'Fraud', '1', 'F' etc., while a legitimate even could be indicated as 'legit', 'Legit', '0', 'L' etc. We have to use exactly the same notation with our CSV file, when creating our labels.

In [ ]:
# collect existing labels
response = fraud_detector_client.get_labels()
existing_label_names = [label['name'] for label in response['labels']]

# create label for fraud
if LABEL_FRAUD not in existing_label_names:
    fraud_detector_client.put_label(
        name = LABEL_FRAUD,
        description = 'label for fraudulent events'
    )
    print('Created label:', LABEL_FRAUD)
else: print('Label', LABEL_FRAUD, 'already exists...')

# create label for legit
if LABEL_LEGIT not in existing_label_names:
    fraud_detector_client.put_label(
        name = LABEL_LEGIT,
        description = 'label for legitimate events'
    )
    print('Created label:', LABEL_LEGIT)
else: print('Label', LABEL_LEGIT, 'already exists...')

## Create Entity type
Entity type describes **who** will perform an actibity (event), for which, we would like to analyze for fraudulent activity. In many cases this is a customer.

In [ ]:
# collect existing entity types
response = fraud_detector_client.get_entity_types()
existing_entity_type_names = [entity_type['name'] for entity_type in response['entityTypes']]

if ENTITY_TYPE_NAME not in existing_entity_type_names:
    fraud_detector_client.put_entity_type(
        name = ENTITY_TYPE_NAME,
        description = 'Sample account entity type'
    )
    print('Created entity:', ENTITY_TYPE_NAME)
else: print('Entity', ENTITY_TYPE_NAME, 'already exists...')


## Create Event type
Event is the **activity** that the Entity will perform, which will be analyzed by Amazon Fraud Detector. Once defined, you can build Models and Detectors that evaluate the risk for specific event types. For example, an Event could be a "sample registration" task that a customer would perform. Other Event types could be an online transaction, the submission of a comment/review etc. When we define an Event type, we need to also include the Variables and Labels that we created in our previous step.

In [ ]:
# collect existing event types
response = fraud_detector_client.get_event_types()
existing_event_type_names = [event_type['name'] for event_type in response['eventTypes']]

if EVENT_TYPE_NAME not in existing_event_type_names:
    fraud_detector_client.put_event_type (
        name = EVENT_TYPE_NAME,
        description='Sample event type',
        eventVariables = list_event_variables,
        labels = [
            LABEL_LEGIT, 
            LABEL_FRAUD
        ],
        entityTypes = [ 
            ENTITY_TYPE_NAME 
        ]
    )
    print('Created event:', EVENT_TYPE_NAME)
else: print('Event', EVENT_TYPE_NAME, 'already exists...')

## Create and train a ML model

Amazon Fraud Detector models learn to detect fraud for a specific event type. In Amazon Fraud Detector, you **first create a Model**, which acts as a **container** for your **Model versions**. Each time you train a model, a new version is created. 

A model version is created by calling the ```CreateModel``` and ```CreateModelVersion``` operations. 

- ```CreateModel``` initiates the model, which acts as a container for your model versions. 

- ```CreateModelVersion``` starts the training process, which results in a specific version of the model. A new version of the solution is created each time you call ```CreateModelVersion```. 


In [ ]:
# collect existing models
response = fraud_detector_client.get_models()
existing_model_ids = [ model['modelId'] for model in response['models'] ]

# create model container
if MODEL_ID not in existing_model_ids:
    fraud_detector_client.create_model (
        modelId = MODEL_ID,
        eventTypeName = EVENT_TYPE_NAME,
        modelType = 'ONLINE_FRAUD_INSIGHTS'
    )
    print('Created model:', MODEL_ID)
else: print('Model', MODEL_ID, 'already exists...')

If the ML model already exists and we initiate a new training, it will results in a new version of the model.

In [ ]:
# start training a new model version
fraud_detector_client.create_model_version (
    modelId = MODEL_ID,
    modelType = 'ONLINE_FRAUD_INSIGHTS',
    trainingDataSource = 'EXTERNAL_EVENTS',
    trainingDataSchema = {
        'modelVariables' : list_event_variables,
        'labelSchema' : {
            'labelMapper' : {
                'FRAUD' : [ LABEL_FRAUD ],
                'LEGIT' : [ LABEL_LEGIT ]
            }
        }
    }, 
    externalEventsDetail = {
        'dataLocation' : S3_FILE_LOC,
        'dataAccessRoleArn' : ARN_ROLE
    }
)

Training can take saveral hours, depending on your dataset. In order to know how training progresses, we add a small piece of code that **polls the training process** in regular intervals and displays the progress. 

In [ ]:
import time

%%time

training_state = None
while training_state != "TRAINING_COMPLETE":
    response = fraud_detector_client.get_model_version(
        modelId = MODEL_ID, 
        modelType = "ONLINE_FRAUD_INSIGHTS", 
        modelVersionNumber = '1.0'
    )
    training_state = response['status']
    print("-", end="")
    time.sleep(60)  # poll once every 1 min

print("\nTraining state:", training_state)

Amazon Fraud Detector keeps aside 10% of the data for validation/testing purposes. You can check the performance of the model by requesting the ROC AUC metric.

In [ ]:
# get model report for the trained model

model_report = fraud_detector_client.describe_model_versions(
    modelId = MODEL_ID,
    modelVersionNumber = '1.0',  # or any other version needed
    modelType = 'ONLINE_FRAUD_INSIGHTS'
)

print('ROC AUC: ',model_report['modelVersionDetails'][0]['trainingResult']['trainingMetrics']['auc'])

After reviewing the model performance, we need to **activate the Model** to make it available for use by Detectors, in real-time fraud predictions. Amazon Fraud Detector will deploy the model in **multiple availability zones** for redundancy with **auto-scaling turned on** to ensure the model scales with the number of fraud predictions you are making. To activate the model, call the ```UpdateModelVersionStatus``` API and update the status to ```ACTIVE```.

In [ ]:
# select model status
MODEL_STATUS = 'ACTIVE' 
# MODEL_STATUS = 'INACTIVE'

response = fraud_detector_client.update_model_version_status (
modelId = MODEL_ID,
modelType = 'ONLINE_FRAUD_INSIGHTS',
modelVersionNumber = '1.00',
status = MODEL_STATUS
)


Deployment may take ~10min, so, we also poll regularly, to know how the process progresses.

In [ ]:
%%time

deployment_state = None
while deployment_state != MODEL_STATUS:
    response = fraud_detector_client.get_model_version(
        modelId = MODEL_ID, 
        modelType = "ONLINE_FRAUD_INSIGHTS", 
        modelVersionNumber = '1.0'
    )
    deployment_state = response['status']
    print("-", end="")
    time.sleep(60)  # poll once every 1 min

print("\nDeployment state:", deployment_state)


## Create a Detector

A detector contains the detection logic, such as the models and rules, for a particular event that you want to evaluate for fraud. During a fraud prediction, you will specify the detector that you want to use to evaluate your event. To create a detector, complete the following steps. 

A detector acts as a container for your detector versions.

In [ ]:
# Collect existing detector ids
response = fraud_detector_client.get_detectors()
existing_detector_ids = [detector['detectorId'] for detector in response['detectors']]

# Detector is a container for detector versions.
if DETECTOR_ID not in existing_detector_ids:
    fraud_detector_client.put_detector (
        detectorId = DETECTOR_ID,
        eventTypeName = EVENT_TYPE_NAME
    )
    print('Created detector:', DETECTOR_ID)
else: print('Detector', DETECTOR_ID, 'already exists...')

## Create outcomes
An outcome is the result of a fraud prediction. Create an outcome for each possible fraud prediction result. For example, you may want outcomes to represent risk levels (high_risk, medium_risk, and low_risk) or actions (approve, review). Once created, you can add one or more outcomes to a rule.

In [ ]:
# Collect existing outcomes
response = fraud_detector_client.get_outcomes()
existing_outcome_names = [ outcome['name'] for outcome in response['outcomes'] ]


if OUTCOME_BLOCK not in existing_outcome_names:
    fraud_detector_client.put_outcome(
        name = OUTCOME_BLOCK,
        description = 'this outcome blocks the event'
    )
    print('Created outcome:', OUTCOME_BLOCK)
else: print('Outcome', OUTCOME_BLOCK, 'already exists...')

if OUTCOME_REVIEW not in existing_outcome_names:
    fraud_detector_client.put_outcome(
        name = OUTCOME_REVIEW,
        description = 'this outcome sidelines event for review'
    )
    print('Created outcome:', OUTCOME_REVIEW)
else: print('Outcome', OUTCOME_REVIEW, 'already exists...')


if OUTCOME_APPROVE not in existing_outcome_names:
    fraud_detector_client.put_outcome(
        name = OUTCOME_APPROVE,
        description = 'this outcome approves the event'
    )
    print('Created outcome:', OUTCOME_APPROVE)
else: print('Outcome', OUTCOME_APPROVE, 'already exists...')
    

## Create rules 
A rule is a condition that tells Amazon Fraud Detector how to interpret variable values during a fraud prediction. A rule consists of one or more variables, a logic expression, and one or more outcomes. A **detector must have at least one associated rule**. Rules in a detector are evaluated as part of a fraud prediction. 
 
 Each rule must contain a single expression that captures your business logic. All expressions must evaluate to a Boolean value (true or false) and be less than 4,000 characters in length. If-else type conditions are not supported. All variables used in the expression must be predefined in the evaluated event type. 

In [ ]:
# in order to use the model score with rules, we need to construct a predefined variable name

MODEL_INSIGHTS = MODEL_ID + '_insightscore'  # to be used when getting real-time predictions
MODEL_SCORE = '$' + MODEL_INSIGHTS  # to be used when defining rules


In [ ]:
# Collect existing rules
response = fraud_detector_client.get_rules(detectorId=DETECTOR_ID)
existing_rule_names = [ rule['ruleId'] for rule in response['ruleDetails'] ]


# for high risk cases
if RULE_ID_HIGH_RISK not in existing_rule_names:
    fraud_detector_client.create_rule(
        ruleId = RULE_ID_HIGH_RISK,
        detectorId = DETECTOR_ID,
        expression = MODEL_SCORE + ' > 800',
        language = 'DETECTORPL',
        outcomes = [OUTCOME_BLOCK]
    )
    print('Created rule:', RULE_ID_HIGH_RISK)
else: print('Rule', RULE_ID_HIGH_RISK, 'already exists...')

# for medium risk cases
if RULE_ID_MID_RISK not in existing_rule_names:
    fraud_detector_client.create_rule(
        ruleId = RULE_ID_MID_RISK,
        detectorId = DETECTOR_ID,
        expression = MODEL_SCORE + ' <= 800 and ' + MODEL_SCORE + ' > 500',
        language = 'DETECTORPL',
        outcomes = [OUTCOME_REVIEW]
    )
    print('Created rule:', RULE_ID_MID_RISK)
else: print('Rule', RULE_ID_MID_RISK, 'already exists...')

# for low risk cases
if RULE_ID_LOW_RISK not in existing_rule_names:
    fraud_detector_client.create_rule(
        ruleId = RULE_ID_LOW_RISK,
        detectorId = DETECTOR_ID,
        expression = MODEL_SCORE + ' <= 500',
        language = 'DETECTORPL',
        outcomes = [OUTCOME_APPROVE]
    )
    print('Created rule:', RULE_ID_LOW_RISK)
else: print('Rule', RULE_ID_LOW_RISK, 'already exists...')

You can **update a rule** by calling the ```UpdateRuleVersion``` API. The following example updates the model score thresholds for the rules high_fraud_risk and medium_fraud_risk **from 900 to 950**. 

In [ ]:
fraud_detector_client.update_rule_version(
    rule = {
        'detectorId' : DETECTOR_ID,
        'ruleId' : RULE_ID_HIGH_RISK,
        'ruleVersion' : '1'
    },
    expression = MODEL_SCORE + ' > 900',
    language = 'DETECTORPL',
    outcomes = [OUTCOME_BLOCK]
)

fraud_detector_client.update_rule_version(
    rule = {
        'detectorId' : DETECTOR_ID,
        'ruleId' : RULE_ID_MID_RISK,
        'ruleVersion' : '1'
    },
    expression = MODEL_SCORE + ' <= 900 and ' + MODEL_SCORE + ' > 500',
    language = 'DETECTORPL',
    outcomes = [OUTCOME_REVIEW]
)

## Create a detector version 
A detector version defines the specific models and rules that will be run as part of a fraud prediction. Each detector version has a status of ```DRAFT```, ```ACTIVE```, or ```INACTIVE```. Only one detector version can be in ```ACTIVE``` status at a time. During the ```GetEventPrediction``` request, Amazon Fraud Detector will use the ```ACTIVE``` detector if no ```DetectorVersion``` is specified. 

In [ ]:
fraud_detector_client.create_detector_version(
    detectorId = DETECTOR_ID,
    rules = [{
        'detectorId' : DETECTOR_ID,
        'ruleId' : RULE_ID_HIGH_RISK,
        'ruleVersion' : '1'
    },
    {
        'detectorId' : DETECTOR_ID,
        'ruleId' : RULE_ID_MID_RISK,
        'ruleVersion' : '1'
    },
    {
        'detectorId' : DETECTOR_ID,
        'ruleId' : RULE_ID_LOW_RISK,
        'ruleVersion' : '1'
    }
    ],
    modelVersions = [{
        'modelId' : MODEL_ID,
        'modelType': 'ONLINE_FRAUD_INSIGHTS',
        'modelVersionNumber' : '1.00'
    }],
    ruleExecutionMode = 'FIRST_MATCHED'
    # ruleExecutionMode = 'ALL_MATCHED'
)

# activate the newly created detector
fraud_detector_client.update_detector_version_status(
    detectorId = DETECTOR_ID,
    detectorVersionId = '1',
    status = 'ACTIVE'
)


## Get fraud predictions

Getting one individual prediction (legit case).

In [ ]:
response = fraud_detector_client.get_event_prediction(
    detectorId = DETECTOR_ID,
    eventId = '802454d3-f7d8-482d-97e8-c4b6db9a0428',
    eventTypeName = EVENT_TYPE_NAME,
    eventTimestamp = '2020-07-13T23:18:21Z',
    entities = [{'entityType':ENTITY_TYPE_NAME, 'entityId':'12345'}],
    eventVariables = {
        'email_address' : 'johndoe@exampledomain.com',
        'ip_address' : '1.2.3.4',
        'billing_state': 'TX',
        'user_agent': 'Mozilla/5.0 (iPad; CPU iPad OS 10_3_3 like Mac OS X) AppleWebKit/532.2 (KHTML, like Gecko) CriOS/34.0.827.0 Mobile/13K063 Safari/532.2',
        'billing_postal': '34491',
        'phone_number': '(555)333 - 9246',
        'billing_address': '12351 Amanda Knolls Fake St.'  
    }
)

print('ML fraud prediction [0,1000]:', response['modelScores'][0]['scores'][MODEL_INSIGHTS])
print('Activated rule:', response['ruleResults'][0]['ruleId'])
print('Outcome:', response['ruleResults'][0]['outcomes'][0])

Getting one individual prediction (fraudulent case).

In [ ]:
response = fraud_detector_client.get_event_prediction(
    detectorId = DETECTOR_ID,
    eventId = '802454d3-f7d8-482d-97e8-c4b6db9a0428',
    eventTypeName = EVENT_TYPE_NAME,
    eventTimestamp = '2020-07-13T23:18:21Z',
    entities = [{'entityType':ENTITY_TYPE_NAME, 'entityId':'12345'}],
    eventVariables = {
        'email_address' : 'fake_timothysmith@example.com',
        'ip_address' : '59.157.144.1',
        'billing_state': 'AZ',
        'user_agent': 'Mozilla/5.0 (iPod; U; CPU iPhone OS 3_0 like Mac OS X; sid-ET) AppleWebKit/534.17.7 (KHTML, like Gecko) Version/4.0.5 Mobile/8B118 Safari/6534.17.7',
        'billing_postal': '32931',
        'phone_number': '(555)596 - 5579',
        'billing_address': '65898 Amy Estate Fake St.'  
    }
)

print('ML fraud prediction [0,1000]:', response['modelScores'][0]['scores'][MODEL_INSIGHTS])
print('Activated rule:', response['ruleResults'][0]['ruleId'])
print('Outcome:', response['ruleResults'][0]['outcomes'][0])

## Deleting resources

Deployed ML models incure cost per hour. It is important to undeploy them, if you don't need them. If the ML model is part of an active Detector, that detector needs to be deactivated first and then to undeploy the ML model. 

### Deleting Detector and Rules

In [ ]:
detectors = fraud_detector_client.get_detectors()
list_of_detectors = [detector['detectorId'] for detector in detectors['detectors']]


if DETECTOR_ID in list_of_detectors:
    # delete detector versions, rules and detector
    response = fraud_detector_client.describe_detector(detectorId = DETECTOR_ID)

    for detector_summary in response['detectorVersionSummaries']:
        if detector_summary['status'] == 'ACTIVE':
            fraud_detector_client.update_detector_version_status(
                detectorId = DETECTOR_ID,
                detectorVersionId = '1',
                status = 'INACTIVE'
            )
            print('Deacivating detector', DETECTOR_ID)
        result = fraud_detector_client.delete_detector_version(
            detectorId = DETECTOR_ID,
            detectorVersionId = detector_summary['detectorVersionId']
        )

    result = fraud_detector_client.get_rules( detectorId = DETECTOR_ID )
    for rule in result['ruleDetails']:
        response = fraud_detector_client.delete_rule(
            rule={
                'detectorId': rule['detectorId'],
                'ruleId': rule['ruleId'],
                'ruleVersion': rule['ruleVersion']
            }
        )
        print('Deleting rule', rule)

    fraud_detector_client.delete_detector(detectorId = DETECTOR_ID)
    print('Deleting detector', DETECTOR_ID)
else:
    print('Detector', DETECTOR_ID, 'not available...')

### Undeploying ML model
ML models and model versions cannot be deleted.

In [ ]:
model_report = fraud_detector_client.describe_model_versions(
        modelId = MODEL_ID,
        modelType='ONLINE_FRAUD_INSIGHTS'
    )

if model_report['modelVersionDetails'][0]['status'] == 'ACTIVE':
    fraud_detector_client.update_model_version_status (
        modelId = MODEL_ID,
        modelType = 'ONLINE_FRAUD_INSIGHTS',
        modelVersionNumber = model_report['modelVersionDetails'][0]['modelVersionNumber'],
        status = 'INACTIVE'
    )
    print('Undeployed model', MODEL_ID)
else: print('Model', MODEL_ID, 'is INACTIVE...')